# Outline

In this file we construct the convex polytopal region denoted $\overline{R_f}$ for FCJ.

Which are intersections of the form $$b\le Ax \le b+1.$$

We aim to determine the number of cases where $R_f = \emptyset$. 

We will check this for the cases: $$G62,G63,G72,G73$$

In [2]:
import numpy as np
import itertools
import pickle

In [3]:
def get_formatted_hyperplanes_txt(lbm,phi_G): #put into format of sage b<Ax<b+1
    
    """
    Inputs:
    phi  records indexs of cuts with cutfraction in inequalities.        phi_G62={"1":1,"2":1.5,"3":1,"4":1,"5":1,"6":1.5,"12":1.5,"23":1.5,"34":1,"45":1,"56":1.5,"61":1.5,"123":1.5,"234":1.5,"345":1,"456":1.5,"561":1.5,"612":1}
    lbm= line bundle mulidegrees given by chip-adding
    
    Purpose: Get list of inquality data to feed into Sages Polyhedron function
    https://doc.sagemath.org/html/en/reference/discrete_geometry/sage/geometry/polyhedron/constructor.html
    
    Polyhedron(ieqs=[(0,1,0),(0,0,1),(1,-1,-1)]).Hrepresentation()
    (An inequality (-1, -1) x + 1 >= 0,
     An inequality (1, 0) x + 0 >= 0,
     An inequality (0, 1) x + 0 >= 0)
    
    
    """
    n=len(lbm[0])
    # We add the total degree equation to inequals
    deg=sum(lbm[0])
    tot_equ1=[-deg,]+ [1]*n
    tot_equ2=[deg,]+ [-1]*n
    
    inequals=[tot_equ1,tot_equ2]         
    for d in lbm: #creates row at a time
        for indexor in list(phi_G.keys()):
            
            def sum_comps(comp,indexor): #Done
                """returns the sum of the components of comp in the position by indexor"""
                #for d and phi
                sum_l=[comp[int(i)-1] for i in indexor]
                return sum(sum_l)
            
            
            cf=phi_G[indexor] # want value
            constant_indx= sum_comps(d,indexor)        
        
            l_i= [ [int(x)-1,] for x in indexor] #x,y,z.. terms for indexors in Rf space #Indexs for phi keys
            

            def lower_part(constant_indx,l_i,indexor,cf,n):
                
                bterm=[constant_indx+cf] # as list
                base=np.zeros(n)
                l=[(-1,i) for i in indexor] # put -1 in the ith position given be indexor.
                
                for item in l:
                    ipos = int(item[1])-1 # ith position
                    val=item[0]
                    base[ipos]=val
                
                xyz=base
                
                ineq=bterm+list(xyz) # [b,x,y,z...] want to concat
                
                return ineq

            def upper_part(constant_indx,l_i,indexor,cf,n):
                
                #Remember upper part of modulus inequality.

                bterm=[-constant_indx+cf] # as list
                base=np.zeros(n)
                l=[(1,i) for i in indexor] # put -1 in the ith position given be indexor.
                
                for item in l:
                    ipos = int(item[1])-1 # ith position
                    val=item[0]
                    base[ipos]=val
                
                xyz=base
                
                ineq=bterm+list(xyz) # [b,x,y,z...] want to concat
                
                return ineq

            res_lower=lower_part(constant_indx,l_i,indexor,cf,n)
            
            # print("res_lower",res_lower)
            res_upper=upper_part(constant_indx,l_i,indexor,cf,n)
            
            # print("res_upper",res_upper)
            res_combined= [res_lower,res_upper]
            
            # print("--------")

            inequals=inequals+res_combined
        
    return inequals

In [4]:
def poly_size(data):
    Poly=Polyhedron(ieqs=data)
    dimP=dim(Poly)
#     print(f"The dimension of the polytope is: {dimP}")
    return dimP

In [5]:
def polytope_size_checker(data,phi,size):
    
    """
    Inputs:
    data: is a list of line bundle multidegrees for the weak stabiltiies that fail when phi is taken to be the average.
    phi= is the data of the cuts and the indexes taken for the phi inequalities.
    size: the size of the nondegenerate polytope (number of vertices)
    """
    recorder_minus=[]
    recorder_0=[]
    recorder_1belowtop=[]
    recorder_max=[]
    dimensions=[]

    counter=0
    for lbm in data:
        lbm_data=get_formatted_hyperplanes_txt(lbm,phi)
        dimp=poly_size(lbm_data)
        if dimp ==0:
            recorder.append(lbm)
        if dimp >0 and dimp<size:
            recorder_1belowtop.append(lbm)
        if dimp ==size:
            recorder_max.append(lbm)
        if dimp==-1:
            recorder_minus.append(lbm)
            
        print(counter)
        dimensions.append(dimp)
        counter+=1

    print("Total number of cases:",len(data))
    print("recorder_minus",len(recorder_minus))
    print("recorder_0:",len(recorder_0))
    print("recorder_1belowtop:",len(recorder_1belowtop))
    print("recorder_max:",len(recorder_max))
    
    print("\n", f"Number of cases of dim: -1:{dimensions.count(-1)}, 0: {dimensions.count(0)}, 1:{dimensions.count(1)}, 2:{dimensions.count(2)}, 3:{dimensions.count(3)}, 4:{dimensions.count(4)}, 5:{dimensions.count(5)}, 6:{dimensions.count(6)}, 7:{dimensions.count(7)}")
          
    return recorder_minus,recorder_0,recorder_1belowtop,recorder_max


In [6]:
def mod_where_specific_lbm_fail(dict,lbm,phi):    
    """
    Checks if a given phi (not average phi) fails.
    
    Outputs where it fails or outputs f"The phi: {} does not fail".
    """
    
    def sum_comps(comp,indexor): #Done
        """returns the sum of the components of comp in the position by indexor"""
        #for d and phi

        sum_l=[comp[int(i)-1] for i in indexor]
        return sum(sum_l)

    def cutfrac(dict,indexor): #Done
        cutfrac=dict[indexor]
        return cutfrac
    
    counter=0
    for d in lbm:
        for indexor in dict.keys():
            if abs(sum_comps(d,indexor) - sum_comps(phi,indexor))<cutfrac(dict,indexor):
                pass
            else:
                print(f"Multidegree: {d}, \n Phi: {phi}, \n Indices summed {indexor}, \n Failed the inequality |{sum_comps(d,indexor)} - {sum_comps(phi,indexor)}|<{cutfrac(dict,indexor)} \n")
                counter+=1
                
    if counter !=0:
        x=f"The phi: {phi} fails the phi-inequalities"
    if counter ==0:
        x=f"The phi: {phi} does not fail the phi-inequalities"
    
    return x


In [7]:
def get_a_phi(poly):
#     phi=poly.center()
    phi=poly.representative_point()
    return phi

# Example test case

In [8]:
"We take G455 and an example set of line bundle multidegrees."

lbm=np.array([[ 0.,  0.,  1.,  1.],
       [ 0.,  1.,  0.,  1.],
       [ 0.,  1.,  1.,  0.],
       [ 0.,  2.,  0.,  0.],
       [ 1.,  0.,  0.,  1.],
       [ 1.,  0.,  1.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  1., -1.]])

phi_G455={"1":1,"2":1.5,"3":1,"4":1.5,"12":1.5,"23":1.5,"34":1.5,"41":1.5}
data=get_formatted_hyperplanes_txt(lbm,phi_G455)
tup=poly_size(data)
# print(tup) #To determine size of data

poly=Polyhedron(ieqs=data)
phi=get_a_phi(poly)
print(phi)

# print(mod_where_specific_lbm_fail(phi_G455,lbm,phi))

(0.5, 0.75, 0.5, 0.25)


# G62

In [9]:
pickle_in = open(r"lbm_data\G62_phi_failcases_lbm.txt", "rb")
data = pickle.load(pickle_in)
pickle_in.close()
print(len(data))


48


In [10]:
"We check if another phi works"

# print(data[0]) #certain the first element is that of Integral break divisors.
lbm=data[0]
# print(lbm)

phi_G62={"1":1,"2":1.5,"3":1,"4":1,"5":1,"6":1.5,"12":1.5,"23":1.5,"34":1,"45":1,"56":1.5,"61":1.5,"123":1.5,"234":1.5,"345":1,"456":1.5,"561":1.5,"612":1}

# tup=polytope_size_checker(data,phi_G62,6)

hyperplanes=get_formatted_hyperplanes_txt(lbm,phi_G62)
# tup=poly_size(data) #should be 6 yup
# print(tup) #To determine size of data

poly=Polyhedron(ieqs=hyperplanes)
phi=get_a_phi(poly)
print(phi) #This is the center of the polytope 
# (compare to previous phi_av: (0.35714286, 0.5,0.21428571, 0.21428571, 0.21428571, 0.5)
# old center: (0.20833333333333331, 0.7083333333333333, 0.20833333333333331, 0.20833333333333331, 0.20833333333333331, 0.7083333333333333)
# new centr: (0.16666666666666666, 0.6666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.6666666666666666)


# # print(mod_where_specific_lbm_fail(phi_G455,lbm,phi))

(0.16666666666666666, 0.6666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.6666666666666666)


In [11]:
pickle_in = open(r"lbm_data\G62_phi_failcases_lbm.txt", "rb")
data = pickle.load(pickle_in)
pickle_in.close()

print(len(data))

# print(data[1])

phi_G62={"1":1,"2":1.5,"3":1,"4":1,"5":1,"6":1.5,"12":1.5,"23":1.5,"34":1,"45":1,"56":1.5,"61":1.5,"123":1.5,"234":1.5,"345":1,"456":1.5,"561":1.5,"612":1}

tup=polytope_size_checker(data,phi_G62,6)


48
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
Total number of cases: 48
recorder_minus 0
recorder_0: 0
recorder_1belowtop: 48
recorder_max: 0

 Number of cases of dim: -1:0, 0: 0, 1:0, 2:0, 3:0, 4:0, 5:48, 6:0, 7:0


# G63

In [12]:
phi_G63={"1":1,"2":1,"3":1.5,"4":1,"5":1,"6":1.5,"12":1,"23":1.5,"34":1.5,"45":1,"56":1.5,"61":1.5,"123":1.5,"234":1.5,"345":1.5,"456":1.5,"561":1.5,"612":1.5}

pickle_in = open(r"lbm_data\G63_phi_failcases_lbm.txt", "rb")
data = pickle.load(pickle_in)
pickle_in.close()

print(len(data))

tup=polytope_size_checker(data,phi_G63,6)


36
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
Total number of cases: 36
recorder_minus 0
recorder_0: 0
recorder_1belowtop: 36
recorder_max: 0

 Number of cases of dim: -1:0, 0: 0, 1:0, 2:0, 3:0, 4:0, 5:36, 6:0, 7:0


# G72

In [8]:
phi_G72={"1":1,"2":1.5,"3":1,"4":1,"5":1,"6":1,"7":1.5,"12":1.5,"23":1.5,"34":1,"45":1,"56":1,"67":1.5,"71":1.5,"123":1.5,"234":1.5,"345":1,"456":1,"567":1.5,"671":1.5,"712":1}

pickle_in = open(r"lbm_data\G72_phi_failcases_lbm.txt", "rb")
data = pickle.load(pickle_in)
pickle_in.close()

print(len(data))
tup=polytope_size_checker(data,phi_G72,7)

240
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
Total number of cases: 240
recorder_minus 0
recorder_0: 0
recorder_1belowtop: 0
recorder_max: 240

 Number of cases of dim: -1:0, 0: 0, 1:0, 2:0, 

In [7]:
#Here we store those with dim =-1
dim_empty_lbm=tup[0]
print(len(dim_empty_lbm))

with open('G72_dim_empty_lbm', 'wb') as fp:
    pickle.dump(dim_empty_lbm, fp)

NameError: name 'tup' is not defined

# G73

In [9]:
phi_G73={"1":1,"2":1,"3":1.5,"4":1,"5":1,"6":1,"7":1.5,"12":1,"23":1.5,"34":1.5,"45":1,"56":1,"67":1.5,"71":1.5,"123":1.5,"234":1.5,"345":1.5,"456":1,"567":1.5,"671":1.5,"712":1.5}

pickle_in = open(r"lbm_data\G73_phi_failcases_lbm.txt", "rb")
data = pickle.load(pickle_in)
pickle_in.close()

print(len(data))
tup73=polytope_size_checker(data,phi_G73,7)

432
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
27

In [46]:
#Here we store those with dim =-1
dim_empty_lbm=tup73[0]
print(len(dim_empty_lbm))

with open('G73_dim_empty_lbm', 'wb') as fp:
    pickle.dump(dim_empty_lbm, fp)

28
